## Clustering graph

In [1]:
# import pandas as pd
# from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt
# sse = []
#
# for i in range(1, 21):
#     # create a KMeans model with the current number of clusters
#     kmeans = KMeans(n_clusters=i)
#     # fit the model to the locations dataframe
#     kmeans.fit(sorted_data_stops[['current_lat', 'current_lng']])
#     # append the sum of squared errors to the list
#     sse.append(kmeans.inertia_)
#
# # create a plot with the number of clusters on the x-axis and SSE on the y-axis
# plt.plot(range(1, 21), sse)
# plt.xlabel('Number of Clusters')
# plt.ylabel('Sum of Squared Errors (SSE)')
# plt.title('Elbow Plot')
# plt.show()

# Artificial routes

In [2]:
# routes[['planned_route_craft', 'actual_route_location']]
#
# import itertools
# artificial_completed_routes = pd.DataFrame(columns=['routes'])
#
# def common_subsequence(planned, actual):
#     return [x[0] for x in itertools.takewhile(lambda x: x[0] == x[1], zip(planned, actual))]
#
# artificial_completed_routes['routes'] = routes.apply(lambda row: common_subsequence(row['planned_route_craft'], row['actual_route_location']), axis=1)
#
# filtered_results = artificial_completed_routes[artificial_completed_routes['routes'].apply(lambda x: len(x) > 1 and len(x) != len(routes.loc[artificial_completed_routes.index, 'planned_route_craft']))]
#
# filtered_results['driver_id'] = routes.loc[filtered_results.index, 'driver_id']
# filtered_results['last_two_weeks_count'] = routes.loc[filtered_results.index, 'last_two_weeks_count']
#
# filtered_results['distance_route'] = filtered_results.apply(lambda row: routes.loc[row.name, 'distance_route'][:len(row['routes'])-1], axis=1)
#
# artificial_planned_routes = pd.DataFrame({'common_subsequence': filtered_results['routes'], 'driver_id': filtered_results['driver_id'], 'distance_route': filtered_results['distance_route'], 'last_two_weeks_count': filtered_results['last_two_weeks_count']})
#
# artificial_planned_routes

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.cluster import KMeans

data_stops = pd.read_csv('uni_molde_v2.csv', sep=',')

data_stops
data_stops.loc[data_stops['stop_completed_at'].isna(), 'stop_completed_at'] = "-1"
data_stops.loc[data_stops['stop_arrived_at'].isna(), 'stop_arrived_at'] = "-1"

print(data_stops.isnull().sum())


sorted_data_stops = data_stops.sort_values(by='stop_dispatched_at', ascending=True)
sorted_data_stops = sorted_data_stops.reset_index(drop=True)
sorted_data_stops['day_of_week'] = pd.to_datetime(sorted_data_stops['stop_dispatched_at']).dt.day_name()
sorted_data_stops['date'] = pd.to_datetime(sorted_data_stops['stop_dispatched_at']).dt.date

#clustering
locations_df = sorted_data_stops[['current_lat', 'current_lng']]
kmeans = KMeans(n_clusters=1000, random_state=42)
kmeans.fit(locations_df)
sorted_data_stops['cluster'] = kmeans.labels_ + 1

sorted_data_stops['location_id_craft'] = sorted_data_stops.groupby(['current_lat', 'current_lng']).ngroup()+1
with open('output.txt', 'w') as f:
    print(sorted_data_stops.to_string(), file=f)
print('number of groups', sorted_data_stops['location_id_craft'].nunique())

# data_stops_day= sorted_data_stops[sorted_data_stops['day_of_week'] == "Wednesday"]
grouped_df = sorted_data_stops.groupby('driver_workday_id')[['driver_id', 'location_type_id', 'address_id', 'stop_dispatched_at', 'stop_arrived_at', 'stop_earliest', 'stop_latest', 'current_lat', 'current_lng', 'stop_completed_at', 'cluster', 'location_id_craft','day_of_week', 'date']].apply(lambda x: pd.Series({
    'driver_id': x['driver_id'].tolist(),
    'location_type_id': x['location_type_id'].tolist(),
    'planned_route_location': x['address_id'].tolist(),
    'stop_dispatched_at': x['stop_dispatched_at'].tolist(),
    'stop_arrived_at': x['stop_arrived_at'].tolist(),
    'stop_earliest': x['stop_earliest'].tolist(),
    'stop_latest': x['stop_latest'].tolist(),
    'current_lat': x['current_lat'].tolist(),
    'current_lng': x['current_lng'].tolist(),
    'stop_completed_at': x['stop_completed_at'].tolist(),
    'planned_route_cluster': x['cluster'].tolist(),
    'planned_route_craft': x['location_id_craft'].tolist(),
    'day_of_week': x['day_of_week'].tolist(),
    'date': x['date'].tolist()
})).reset_index()

stop_completed_at        0
stop_arrived_at          0
stop_latest              0
stop_earliest            0
stop_dispatched_at       0
location_id              0
location_type_id         0
driver_workday_id        0
organization_id          0
address_id               0
location_is_depot        0
driver_id                0
contact_id               0
current_lat              0
current_lng              0
prev_planned_lat      7450
prev_planned_lng      7450
prev_actual_lat       7450
prev_actual_lng       7450
dtype: int64
number of groups 3110


In [17]:
import folium
import math

lat_seq = grouped_df['current_lat'][0]
lon_seq = grouped_df['current_lng'][0]

print(lat_seq, lon_seq)
def build_route(lat_seq, lon_seq):
    m = folium.Map(location=[lat_seq[0], lon_seq[0]], zoom_start=13)

    # Add a polyline with arrows
    for i in range(len(lat_seq)):
        lat, lon = lat_seq[i], lon_seq[i]

        # Add an index label
        folium.Marker([lat, lon], icon=folium.DivIcon(html=f'<div style="font-size: 13pt; border: 2px solid white; border-radius: 50%; padding: 2px; background-color: white">{i+1}</div>')).add_to(m)

        if i < len(lat_seq) - 1:
            next_lat, next_lon = lat_seq[i + 1], lon_seq[i + 1]
            angle = np.rad2deg(np.arctan2(next_lat - lat, next_lon - lon))
            arrow_lon = lon + 0.001 * np.cos(np.deg2rad(angle))

            # Add a line segment
            folium.PolyLine([[lat, lon], [next_lat, next_lon]], color='blue', dash_array='5, 5').add_to(m)

            # Add an arrow at the end of the line segment
            folium.Marker([next_lat, next_lon], icon=folium.Icon(color='blue', icon='arrow-up', prefix='fa')).add_to(m)

    m.save('route_map.html')

[55.647, 55.667, 55.677, 55.676, 55.671, 55.666, 55.669, 55.667, 55.666, 55.654] [12.152, 12.526, 12.566, 12.562, 12.561, 12.565, 12.57, 12.567, 12.565, 12.545]
